# LangGraph Stream Parser - Jupyter Display Demo

This notebook demonstrates the value of `JupyterDisplay` for visualizing LangGraph agent streams.

## The Problem

When streaming from a LangGraph agent, the raw output is:
- **Verbose** - Nested dictionaries with metadata you don't care about
- **Hard to follow** - Tool calls, messages, and interrupts all mixed together
- **Not real-time friendly** - Output floods the cell instead of updating in place

## The Solution

`JupyterDisplay` provides:
- **Clean panels** - Content, tools, and interrupts in organized boxes
- **Live updates** - Updates in place instead of flooding output
- **Tool lifecycle** - See tools transition from running → success/error with timing
- **Prominent interrupts** - Human-in-the-loop requests are impossible to miss

## Setup

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from agent import agent

---
## Before: Raw Streaming Output

This is what you get with vanilla `graph.stream()` - a flood of nested dictionaries:

In [3]:
# Raw streaming - hard to follow!
for chunk in agent.stream(
    dict(messages=["What files are in the current directory?"]), 
    config=dict(configurable=dict(thread_id="raw-demo")),
    stream_mode="updates"
):
    print(chunk)
    print("---")

{'PatchToolCallsMiddleware.before_agent': {'messages': Overwrite(value=[HumanMessage(content='What files are in the current directory?', additional_kwargs={}, response_metadata={}, id='844c9f3c-eea8-4da1-9566-6f8e586094e3')])}}
---
{'SummarizationMiddleware.before_model': None}
---{'model': {'messages': [AIMessage(content=[{'text': "I'll list the files in the current directory for you.", 'type': 'text'}, {'id': 'toolu_014agEhbJ7Cpg2fjR85UV4g4', 'input': {'path': '/'}, 'name': 'ls', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_016FoMo9KtBfUdwJH8tyCYkC', 'model': 'claude-sonnet-4-5-20250929', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0}, 'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 6258, 'input_tokens': 3, 'output_tokens': 64, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-sonnet-4-5-20250929', 'model_provider': 'anthr

---
## After: JupyterDisplay

Same stream, but with clean, live-updating panels:

In [4]:
from langgraph_stream_parser.adapters.jupyter import JupyterDisplay

In [5]:
display = JupyterDisplay()

display.stream(
    agent.stream(
        dict(messages=["What files are in the current directory?"]), 
        config=dict(configurable=dict(thread_id="display-demo")),
        stream_mode="updates"
    )
)

╭─────────────────────────────────────────── Assistant ────────────────────────────────────────────╮
│ I'll list the files in the current directory for you.The current directory contains:             │
│                                                                                                  │
│ 1. **`.ipynb_checkpoints/`** - Directory for Jupyter notebook checkpoints                        │
│ 2. **`__pycache__/`** - Directory for Python compiled bytecode files                             │
│ 3. **`agent.py`** - A Python script                                                              │
│ 4. **`jupyter_display.py`** - A Python script for Jupyter display functionality                  │
│ 5. **`jupyter_example.ipynb`** - A Jupyter notebook file                                         │
│                                                                                                  │
│ Would you like me to explore any of these files in more detail?                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── Tools ──────────────────────────────────────────────╮
│                                                                                                  │
│   Status     Tool   Args           Time                                                          │
│  ───────────────────────────────────────                                                         │
│   OK         ls     path=/         33ms                                                          │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

Stream completed

In [6]:
display = JupyterDisplay()

display.stream(
    agent.stream(
        dict(messages=["what's in the third one?"]), 
        config=dict(configurable=dict(thread_id="display-demo")),
        stream_mode="updates"
    )
)

╭─────────────────────────────────────────── Assistant ────────────────────────────────────────────╮
│ I'll read the contents of `jupyter_example.ipynb` for you.The file `jupyter_example.ipynb` is a  │
│ Jupyter notebook that demonstrates the **JupyterDisplay** tool for visualizing LangGraph agent   │
│ streams.                                                                                         │
│                                                                                                  │
│ Here's what it contains:                                                                         │
│                                                                                                  │
│ **Purpose:**                                                                                     │
│ - It showcases how `JupyterDisplay` helps visualize LangGraph agent streams in a cleaner, more   │
│ organized way compared to raw streaming output                                                   │
│                                                                                                  │
│ **Key sections:**                                                                                │
│                                                                                                  │
│ 1. **Introduction** - Explains the problem with raw LangGraph streaming output (verbose, hard to │
│ follow, not real-time friendly)                                                                  │
│                                                                                                  │
│ 2. **The Solution** - Describes how `JupyterDisplay` provides:                                   │
│    - Clean panels for content, tools, and interrupts                                             │
│    - Live updates in place                                                                       │
│    - Tool lifecycle tracking                                                                     │
│    - Prominent interrupt displays                                                                │
│                                                                                                  │
│ 3. **Setup** - Imports the necessary modules including:                                          │
│    - Loading environment variables with `dotenv`                                                 │
│    - Importing the agent from `agent.py`                                                         │
│                                                                                                  │
│ 4. **Before Example** - Shows raw streaming output from `agent.stream()` with a demonstration    │
│ query "What files are in the current directory?" The output shows the typical verbose nested     │
│ dictionaries that are hard to parse.                                                             │
│                                                                                                  │
│ The notebook appears to be a tutorial/demo showing the difference between raw LangGraph          │
│ streaming and using the JupyterDisplay enhancement.                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── Tools ──────────────────────────────────────────────╮
│                                                                                                  │
│   Status     Tool        Args                                     Time                           │
│  ──────────────────────────────────────────────────────────────────────                          │
│   OK         read_file   file_path=/jupyter_example.ipynb         11ms                           │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

Stream completed

---
## Key Features

### 1. Tool Lifecycle Tracking

Tools show their status in real-time:
- `⏳ ...` - Running
- `✅ OK` - Success (with execution time)
- `❌ ERR` - Error

### 2. Interrupt Handling

When the agent needs approval (like for `bash` commands), you get a prominent red panel showing:
- The tool requesting approval
- The arguments it wants to use
- Allowed decisions (approve/reject/edit)

### 3. Content Accumulation

Assistant responses stream smoothly in a blue panel instead of chunk-by-chunk spam.

---
## Manual Event Processing

For more control, you can process events individually:

In [ ]:
from langgraph_stream_parser import StreamParser
from langgraph_stream_parser.events import InterruptEvent, ContentEvent, ToolCallStartEvent

parser = StreamParser()
display = JupyterDisplay()

for event in parser.parse(
    agent.stream(
        dict(messages=["Count Python files in this directory"]),
        config=dict(configurable=dict(thread_id="manual-demo")),
        stream_mode="updates"
    )
):
    display.update(event)
    
    # You can also handle events programmatically
    if isinstance(event, InterruptEvent):
        print(f"\n[Interrupt detected - {len(event.action_requests)} action(s) need approval]")

---
## Configuration Options

In [ ]:
# Customize the display
display = JupyterDisplay(
    show_timestamps=False,     # Show/hide timestamps
    show_tool_args=True,       # Show tool arguments in the status table
    max_content_preview=200,   # Max characters for extraction previews
)

---
## Summary

| Raw Streaming | JupyterDisplay |
|---------------|----------------|
| Nested dicts flood output | Clean, organized panels |
| Tool status buried in data | Visual tool lifecycle (⏳→✅/❌) |
| Interrupts easy to miss | Prominent red interrupt panel |
| Content comes in chunks | Smooth content accumulation |
| No timing information | Execution time per tool |